##### Copyright 2021 The TF-Agents Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# ネットワーク

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/agents/tutorials/8_networks_tutorial"><img src="https://www.tensorflow.org/images/tf_logo_32px.png"> TensorFlow.orgで表示</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ja/agents/tutorials/8_networks_tutorial.ipynb">     <img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colabで実行</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ja/agents/tutorials/8_networks_tutorial.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub でソースを表示{</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ja/agents/tutorials/8_networks_tutorial.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">ノートブックをダウンロード/a0}</a></td>
</table>

## はじめに

このコラボでは、エージェントにカスタムネットワークを定義する方法について説明します。ネットワークはエージェントによってトレーニングされるモデルを定義するのに役立ちます。TF-Agents では、エージェント間で役に立つ複数種類のネットワークがあります。

**メインネットワーク**

- **QNetwork**: このネットワークは別々のアクションを持つ環境の Qlearning で使用され、観測をマッピングしてそれぞれの可能なアクションの推定値を評価します。
- **CriticNetworks**: 文献では `ValueNetworks` とも呼ばれており、いくつかの状態をあるポリシーの期待リターンの推定値にマッピングする Value 関数のいくつかのバージョンを推定することを学習します。これらのネットワークは、エージェントが現在どの程度良好な状態にあるのかを推定します。
- **ActorNetworks**: 観測からアクションへのマッピングを学習します。このネットワークは通常、アクションを生成するためにポリシーによって使用されます。
- **ActorDistributionNetworks**: `ActorNetworks` に似ていますが、ポリシーがサンプリングしてアクションを生成できる分布を生成します。

**ヘルパーネットワーク**

- **EncodingNetwork**: ユーザーがネットワークの入力に適用する前処理レイヤーのマッピングを簡単に定義できるようにします。
- **DynamicUnrollLayer**: 時系列に適用されると、エピソード境界でネットワークの状態を自動的にリセットします。
- **ProjectionNetwork**: `CategoricalProjectionNetwork` や `NormalProjectionNetwork` のようなネットワークは入力を受け付け、カテゴリカル分布や正規分布の生成に必要なパブリックベータを生成します。

TF-Agents のすべての例にはネットワークが事前に構成されています。ただし、それらのネットワークは複雑な観測を処理するようにはセットアップされていません。

複数の観測/アクションを公開する環境があり、ネットワークのカスタマイズが必要な方にはこのチュートリアルが適しています。

## セットアップ

tf-agents をまだインストールしていない場合は、次を実行します。

In [ ]:
!pip install tf-agents

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import abc
import tensorflow as tf
import numpy as np

from tf_agents.environments import random_py_environment
from tf_agents.environments import tf_py_environment
from tf_agents.networks import encoding_network
from tf_agents.networks import network
from tf_agents.networks import utils
from tf_agents.specs import array_spec
from tf_agents.utils import common as common_utils
from tf_agents.utils import nest_utils

tf.compat.v1.enable_v2_behavior()

## ネットワークの定義

### ネットワーク API

TF-Agents では Keras [ネットワーク](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/keras/engine/network.py)からサブクラスを作っています。これを使用して以下を実行できます。

- ターゲットネットワークの作成時に必要なコピー操作を単純化する。
- `network.variables()` の呼び出し時に自動変数の作成を実行する。
- ネットワークの input_specs を基に入力を検証する。

##EncodingNetwork 上述のように `EncodingNetwork` を使用すると、ネットワークの入力に適用する前処理レイヤーのマッピングを簡単に定義していくつかのエンコードを生成することができます。

EncodingNetwork は、次のほとんどのオプションレイヤーから構成されています。

- 前処理レイヤー
- 前処理コンバイナー
- Conv2D
- Flatten
- Dense

エンコーディングネットワークの特徴は、入力の前処理が適用されることです。入力の前処理は、`preprocessing_layers` および `preprocessing_combiner` レイヤーを介して実行可能です。これらはそれぞれネスト構造として指定できます。`preprocessing_layers` ネストが `input_tensor_spec` よりも浅い場合、これらのレイヤーはサブネストを取得します。例を以下に示します。

```
input_tensor_spec = ([TensorSpec(3)] * 2, [TensorSpec(3)] * 5)
preprocessing_layers = (Layer1(), Layer2())
```

この場合、前処理によって以下が呼び出されます。

```
preprocessed = [preprocessing_layers[0](observations[0]),
                preprocessing_layers[1](observations[1])]
```

しかし、次の場合はどうなるでしょうか。

```
preprocessing_layers = ([Layer1() for _ in range(2)],
                        [Layer2() for _ in range(5)])
```

この場合、前処理によって以下が呼び出されます。

```python
preprocessed = [
  layer(obs) for layer, obs in zip(flatten(preprocessing_layers),
                                    flatten(observations))
]
```


### カスタムネットワーク

独自ネットワークを作成するには、`__init__` メソッドと `call` メソッドをオーバーライドするだけです。`EncodingNetworks` について学習した内容を使用してカスタムネットワークを作成し、画像とベクトルを含む観測を取得する ActorNetwork を作成してみましょう。


In [ ]:
class ActorNetwork(network.Network):

  def __init__(self,
               observation_spec,
               action_spec,
               preprocessing_layers=None,
               preprocessing_combiner=None,
               conv_layer_params=None,
               fc_layer_params=(75, 40),
               dropout_layer_params=None,
               activation_fn=tf.keras.activations.relu,
               enable_last_layer_zero_initializer=False,
               name='ActorNetwork'):
    super(ActorNetwork, self).__init__(
        input_tensor_spec=observation_spec, state_spec=(), name=name)

    # For simplicity we will only support a single action float output.
    self._action_spec = action_spec
    flat_action_spec = tf.nest.flatten(action_spec)
    if len(flat_action_spec) > 1:
      raise ValueError('Only a single action is supported by this network')
    self._single_action_spec = flat_action_spec[0]
    if self._single_action_spec.dtype not in [tf.float32, tf.float64]:
      raise ValueError('Only float actions are supported by this network.')

    kernel_initializer = tf.keras.initializers.VarianceScaling(
        scale=1. / 3., mode='fan_in', distribution='uniform')
    self._encoder = encoding_network.EncodingNetwork(
        observation_spec,
        preprocessing_layers=preprocessing_layers,
        preprocessing_combiner=preprocessing_combiner,
        conv_layer_params=conv_layer_params,
        fc_layer_params=fc_layer_params,
        dropout_layer_params=dropout_layer_params,
        activation_fn=activation_fn,
        kernel_initializer=kernel_initializer,
        batch_squash=False)

    initializer = tf.keras.initializers.RandomUniform(
        minval=-0.003, maxval=0.003)

    self._action_projection_layer = tf.keras.layers.Dense(
        flat_action_spec[0].shape.num_elements(),
        activation=tf.keras.activations.tanh,
        kernel_initializer=initializer,
        name='action')

  def call(self, observations, step_type=(), network_state=()):
    outer_rank = nest_utils.get_outer_rank(observations, self.input_tensor_spec)
    # We use batch_squash here in case the observations have a time sequence
    # compoment.
    batch_squash = utils.BatchSquash(outer_rank)
    observations = tf.nest.map_structure(batch_squash.flatten, observations)

    state, network_state = self._encoder(
        observations, step_type=step_type, network_state=network_state)
    actions = self._action_projection_layer(state)
    actions = common_utils.scale_to_spec(actions, self._single_action_spec)
    actions = batch_squash.unflatten(actions)
    return tf.nest.pack_sequence_as(self._action_spec, [actions]), network_state

`RandomPyEnvironment` を作成し、構造化した観測を生成して実装を検証しましょう。

In [ ]:
action_spec = array_spec.BoundedArraySpec((3,), np.float32, minimum=0, maximum=10)
observation_spec =  {
    'image': array_spec.BoundedArraySpec((16, 16, 3), np.float32, minimum=0,
                                        maximum=255),
    'vector': array_spec.BoundedArraySpec((5,), np.float32, minimum=-100,
                                          maximum=100)}

random_env = random_py_environment.RandomPyEnvironment(observation_spec, action_spec=action_spec)

# Convert the environment to a TFEnv to generate tensors.
tf_env = tf_py_environment.TFPyEnvironment(random_env)

観測をディクショナリとして定義しましたので、観測を処理する前処理レイヤーを作成する必要があります。

In [ ]:
preprocessing_layers = {
    'image': tf.keras.models.Sequential([tf.keras.layers.Conv2D(8, 4),
                                        tf.keras.layers.Flatten()]),
    'vector': tf.keras.layers.Dense(5)
    }
preprocessing_combiner = tf.keras.layers.Concatenate(axis=-1)
actor = ActorNetwork(tf_env.observation_spec(), 
                     tf_env.action_spec(),
                     preprocessing_layers=preprocessing_layers,
                     preprocessing_combiner=preprocessing_combiner)

これでアクターネットワークを用意できたので、環境から観測を処理できるようになりました。

In [ ]:
time_step = tf_env.reset()
actor(time_step.observation, time_step.step_type)

これと同じ戦略をエージェントが使用するメインネットワークのカスタマイズに使用できます。任意の前処理を定義し、それをネットワークの残りの部分に接続できます。独自のものを定義する場合は、ネットワークの出力レイヤーの定義が一致していることを確認してください。